<a href="https://colab.research.google.com/github/ShabnaIlmi/Data-Science-Group-Project/blob/Importer_Risk_Prediction/Model_Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Installing required libraries
# !pip install --upgrade tensorflow
# !pip install fancyimpute scikit-learn pandas

In [253]:
# Importing necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.experimental import enable_iterative_imputer
from sklearn.preprocessing import StandardScaler
from sklearn.impute import IterativeImputer, KNNImputer
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.cluster import KMeans
from sklearn.ensemble import RandomForestRegressor
from sklearn.experimental import enable_iterative_imputer
import warnings
from imblearn.over_sampling import SMOTE
from sklearn.impute import IterativeImputer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import GradientBoostingClassifier
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import accuracy_score, classification_report
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import KNNImputer
import random
import pickle
import joblib

In [ ]:
# Mounting the google drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Loading the dataset with the proper delimiter (semicolon)
data = pd.read_excel('/content/drive/MyDrive/Comprehensive_Chemical_Risk_Prediction_Model/IMPORT STATISTICS - 2023.xlsx')

In [ ]:
# Displaying the first few rows of the data
data.head()

In [ ]:
# Displaying information
print("Displaying data information")
data.info()

In [ ]:
# Identifying categorical and numerical columns
categorical_cols = data.select_dtypes(include=['object']).columns
numerical_cols = data.select_dtypes(include=['int64', 'float64']).columns

In [ ]:
# Displaying the Categorical and the Numerical columns'
print("Categorical Columns:")
print(categorical_cols)

print("\nNumerical Columns:")
print(numerical_cols)

In [ ]:
# Converting 'IMPORTER' and 'HSCODE' columns to string type
data['HSCODE'] = data['HSCODE'].astype(str)

# Displaying the HSCODE column data type
print("Data Type of HSCODE Columns:")
print(data[['HSCODE']].dtypes)

In [ ]:
# List of categorical features
categorical_features = ['IMPORTER', 'MONTH', 'COUNTRY', 'HSCODE', 'UNIT', 'DESCRIPTION_01', 'DESCRIPTION_02', 'DESCRIPTION_03']

# Displaying the categorical features
print("Categorical Features:")
for feature in categorical_features:
    print(f"- {feature}")

# Display data type of the columns
print("\nData Type of Categorical Features:")
print(data[categorical_features].dtypes)

In [ ]:
# Displaying the unique values and their counts relevant to each categorical column
print("Unique values and their count relevant to each categorical column:\n")
for col in categorical_features:
    unique_values = data[col].unique()
    value_counts = data[col].value_counts()
    print(value_counts)
    print(" ")

In [ ]:
# Displaying the categorical columns which contains null values and their counts
print("Categorical columns with null values and their counts:")
for col in categorical_features:
    null_count = data[col].isnull().sum()
    if null_count > 0:
        print(f"{col}: {null_count}")

In [ ]:
# Displaying the categorical columns which contain 'Unknown' values and their relevant counts
print("Categorical columns with 'Unknown' values and their counts:")
for col in categorical_features:
    unknown_count = (data[col] == 'Unknown').sum()
    if unknown_count > 0:
        print(f"{col}: {unknown_count}")

In [ ]:
# Converting the 'Year' column to int64 data type
# data['YEAR'] = data['YEAR'].astype(int)

# Displaying the data type of the 'Year' column
# print("Data Type of 'Year' Column:")
# print(data['YEAR'].dtype)

In [ ]:
# List of numerical features
numerical_features = ['YEAR', 'QUANTITY', 'VALUE_RS']

# Displaying the numerical features
print("Numerical Features:")
for feature in numerical_features:
    print(f"- {feature}")

# Display the data type of the numerical features
print("\nData Type of Numerical Features:")
print(data[numerical_features].dtypes)

In [ ]:
# Displaying the unique values and their count in the numerical columns
print("Unique values and their count in the numerical columns:\n")
for col in numerical_features:
    unique_values = data[col].unique()
    value_counts = data[col].value_counts()
    print(value_counts)
    print(" ")

In [ ]:
# Displaying the numerical columns with null values and their relevant counts
print("Numerical columns with null values and their relevant counts:")
for col in numerical_cols:
    null_count = data[col].isnull().sum()
    if null_count > 0:
        print(f"{col}: {null_count}")

In [ ]:
# Step 1: Removing whitespaces from the object type columns
object_columns = data.select_dtypes(include=['object']).columns
data[object_columns] = data[object_columns].apply(lambda x: x.str.strip())

In [ ]:
# Step 2: Removing unnecessary full stops(".") from the categorical columns
data[categorical_cols] = data[categorical_cols].apply(lambda x: x.str.replace('.', ''))

In [ ]:
# Step 3: Removing unnecessary special characters and trailing spaces from the 'DESCRIPTION_03' column
data['DESCRIPTION_03'] = data['DESCRIPTION_03'].str.replace(r'[^\w\s]', '', regex=True)
data['DESCRIPTION_03'] = data['DESCRIPTION_03'].str.strip()

In [ ]:
# Step 4: Removing trailing spaces for the entire 'DESCRIPTION_02' column
data['DESCRIPTION_02'] = data['DESCRIPTION_02'].str.strip()

In [ ]:
# Displaying the unique values and their counts relevant to each categorical column
print("Unique values and their count relevant to each categorical column:\n")
for col in categorical_features:
    unique_values = data[col].unique()
    value_counts = data[col].value_counts()
    print(value_counts)
    print(" ")

In [ ]:
# Step 5: Converting all the values in the 'COUNTRY' and the 'UNIT' columns to uppercase values
data['COUNTRY'] = data['COUNTRY'].str.upper()
data['UNIT'] = data['UNIT'].str.upper()

# Displaying the modified dataset
print(data)

In [ ]:
# Step 6: Converting the 'KGS' values in the 'Unit' columns to 'KG'
data.loc[data['UNIT'] == 'KGS', 'UNIT'] = 'KG'

# Displaying the modified dataset
print(data)

In [ ]:
# Step 7: Handling the HSCODES

# Checking the length of the HSCODES and determining the maximum length
data['HSCODE_LENGTH'] = data['HSCODE'].apply(len)
max_length = data['HSCODE_LENGTH'].max()

# Padding HSCODE values with trailing zeros to match the maximum length
data['HSCODE'] = data['HSCODE'].apply(lambda x: x.ljust(max_length, '0'))

# Dropping the helper column 'HSCODE_LENGTH' as it's no longer needed
data.drop(columns=['HSCODE_LENGTH'], inplace=True)

# Displaying the modified dataset
print(data)

In [ ]:
# Step 8: Handling the 'COUNTRY' Column

# Replacing specific country names
data['COUNTRY'] = data['COUNTRY'].replace({
    'United States': 'USA',
    'United States of America': 'USA',
    'United Kingdom': 'UK',
    'Korea, Republic of': 'South Korea'
})

print(data)

In [ ]:
# Displaying the unique values and their counts relevant to each categorical column
print("Unique values and their count relevant to each categorical column:\n")
for col in categorical_features:
    unique_values = data[col].unique()
    value_counts = data[col].value_counts()
    print(value_counts)
    print(" ")

In [ ]:
# Step 9: Handling missing values in the categorical columns

# Replacing all the missing values in the categorical columns with 'Unknown' for imputation
data[categorical_features] = data[categorical_features].fillna('Unknown')

In [ ]:
# Verifying the changes after replacing the missing values with 'Unknown'
print("Checking for any missing values left behind after replacing with 'Unknown':")
for col in categorical_features:
    null_count = data[col].isnull().sum()
    if null_count > 0:
        print(f"{col}: {null_count} missing values")
    else:
        print(f"{col}: No missing values")
    print(" ")

In [ ]:
# Handling the 'UNknown' values in the 'COUNTRY' column using the mode
mode_country = data['COUNTRY'].mode()[0]
data['COUNTRY'] = data['COUNTRY'].replace('Unknown', mode_country)

In [ ]:
# Displaying dataset information post-imputation
print("\nDataset information after categorical imputation:")
data.info()

In [ ]:
# Displaying the unique values in the 'IMPORTER' column
unique_importers = data['IMPORTER'].unique()
print("Unique Importers:")
print(unique_importers)

In [ ]:
# Display the unique values in the 'YEAR' column
unique_years = data['MONTH'].unique()
print("Unique MONTH:")
print(unique_years)

In [ ]:
# Display unique values in the 'COUNTRY' column
unique_countries = data['COUNTRY'].unique()
print("Unique Countries:")
print(unique_countries)

In [ ]:
# Display unique values in the 'HSCODE' column
unique_hscodes = data['HSCODE'].unique()
print("Unique HSCodes:")
print(unique_hscodes)

In [ ]:
# Display the unique values in the 'UNIT' column
unique_units = data['UNIT'].unique()
print("Unique Units:")
print(unique_units)

In [ ]:
# Displaying the unique values in the 'DESCRIPTION_01' column
unique_descriptions_01 = data['DESCRIPTION_01'].unique()
print("Unique Descriptions_01:")
print(unique_descriptions_01)

In [ ]:
# Displaying the unique values in the 'DESCRIPTION_02' column
unique_descriptions_02 = data['DESCRIPTION_02'].unique()
print("Unique Descriptions_02:")
print(unique_descriptions_02)

In [ ]:
# Displaying the unique values in the 'DESCRIPTION_03' column
unique_descriptions_03 = data['DESCRIPTION_03'].unique()
print("Unique Descriptions_03:")
print(unique_descriptions_03)

In [ ]:
# Step 10: Encoding the Categorical Columns

# List of categorical columns to encode (excluding 'DESCRIPTION_03' which will be handled separately)
encoding_columns = ['IMPORTER', 'MONTH', 'COUNTRY', 'HSCODE', 'UNIT', 'DESCRIPTION_01', 'DESCRIPTION_02']

# Creating a label encoder object
label_encoder = LabelEncoder()

# Encoding and saving columns
for col in encoding_columns:
    # Encoding the column
    data[col] = label_encoder.fit_transform(data[col])

    # If the column is not one of the specific columns, save the encoder
    if col not in ['IMPORTER', 'MONTH']:
        with open(f'encoded_{col}.pkl', 'wb') as f:
            pickle.dump(label_encoder, f)
        print(f"Encoded column: {col} saved successfully")
    else:
        print(f"Encoded column: {col}, but encoder not saved.")

# Separate encoding for 'DESCRIPTION_03'
description_03_encoder = LabelEncoder()
data['ENCODED_DESCRIPTION_03'] = description_03_encoder.fit_transform(data['DESCRIPTION_03'])

# Saving the encoder for 'DESCRIPTION_03'
with open('encoded_DESCRIPTION_03.pkl', 'wb') as f:
    pickle.dump(description_03_encoder, f)

print("Encoded column: DESCRIPTION_03 saved successfully")

In [ ]:
# Displaying the categorical columns after the encoding
print("Categorical columns after encoding:")
print(data[encoding_columns])

In [ ]:
# Imputing the 'MONTH' column with K-NN Imputer

# Initialize the KNNImputer with 5 neighbors
knn_imputer = KNNImputer(n_neighbors=5)

# Applying the imputation to the 'MONTH' column
data['MONTH'] = knn_imputer.fit_transform(data[['MONTH']])

In [ ]:
# Step 11: Handling missing values in the numerical columns

# KNN Imputation for 'QUANTITY'
knn_imputer = KNNImputer(n_neighbors=5)
data[['QUANTITY']] = knn_imputer.fit_transform(data[['QUANTITY']])

In [ ]:
# Preparing the data, using the separately encoded 'ENCODED_DESCRIPTION_03' and ignoring 'DESCRIPTION_03'
missing_value_rs = data[data['VALUE_RS'].isnull()]
non_missing_value_rs = data[data['VALUE_RS'].notnull()]

# Dropping the 'VALUE_RS' and 'DESCRIPTION_03', use 'ENCODED_DESCRIPTION_03' instead
X_value_rs = non_missing_value_rs.drop(columns=['VALUE_RS', 'DESCRIPTION_03'])
X_value_rs['ENCODED_DESCRIPTION_03'] = non_missing_value_rs['ENCODED_DESCRIPTION_03']

# Considering the 'VALUE_RS' as the target variable
y_value_rs = non_missing_value_rs['VALUE_RS']

# Training Random Forest Regressor
rf_regressor = RandomForestRegressor(random_state=42, n_estimators=100)
rf_regressor.fit(X_value_rs, y_value_rs)

# Preparing the missing data using 'ENCODED_DESCRIPTION_03' for prediction
X_missing_value_rs = missing_value_rs.drop(columns=['VALUE_RS', 'DESCRIPTION_03'])
X_missing_value_rs['ENCODED_DESCRIPTION_03'] = missing_value_rs['ENCODED_DESCRIPTION_03']

# Predicting missing 'VALUE_RS' values and impute them back into the original dataframe
data.loc[data['VALUE_RS'].isnull(), 'VALUE_RS'] = rf_regressor.predict(X_missing_value_rs)


In [ ]:
# Iterative Imputation for 'YEAR'
iterative_imputer = IterativeImputer(random_state=42)
data['YEAR'] = iterative_imputer.fit_transform(data[['YEAR', 'MONTH', 'QUANTITY', 'VALUE_RS']])

In [ ]:
# Step 12: Handling the missing values in the 'IMPORTER' Column (Categorical Column)

# Imputing the 'IMPORTER' column using the K-Means Clustering

# Preparing features for clustering
X = data[['COUNTRY', 'HSCODE', 'QUANTITY', 'VALUE_RS']]

# Applying K-Means clustering
kmeans = KMeans(n_clusters=5, random_state=42)  # Adjust n_clusters as needed
data['CLUSTER'] = kmeans.fit_predict(X)

# Assigning importer names based on clusters
importer_names = [
    "Global Traders Inc.", "Eastern Imports Ltd.", "Pacific Exports Co.",
    "Summit Commerce", "EverGreen Enterprises"
]
data['IMPORTER'] = data['CLUSTER'].apply(lambda x: importer_names[x])

# Dropping the helper columns(encoded categorical columns)
data = data.drop(columns=['COUNTRY', 'HSCODE', 'CLUSTER'])

# Displaying the updated dataset
print(data)

In [ ]:
# Step 13: Handling the 'YEAR' Column
# Converting the year data type into int64
data['YEAR'] = data['YEAR'].astype(int)

# Displaying the dataset information
print(data)

In [ ]:
# Step 14: Re-encode the Importer column
label_encoder = LabelEncoder()
data['IMPORTER'] = label_encoder.fit_transform(data['IMPORTER'])

# Saving the encoder for 'IMPORTER' column
with open('encoded_IMPORTER.pkl', 'wb') as f:
    pickle.dump(label_encoder, f)

# Displaying the encoded details
print(data)

In [ ]:
# Step 15: Handling the outliers in the numerical columns

# Creating an empty dictionary to store the outliers
outlier_info = {}

for column in numerical_features:
    Q1, Q3 = data[column].quantile([0.25, 0.75])
    IQR = Q3 - Q1
    lower_bound, upper_bound = Q1 - 1.5 * IQR, Q3 + 1.5 * IQR

    # Detecting outliers
    outliers = data[(data[column] < lower_bound) | (data[column] > upper_bound)]

    outlier_info[column] = {
        'Outliers': outliers.shape[0],
        'Lower Bound': lower_bound,
        'Upper Bound': upper_bound
    }

    print(f"Column: {column}")
    print(f"  Outliers: {outlier_info[column]['Outliers']}, "
          f"Lower: {lower_bound:.2f}, Upper: {upper_bound:.2f}")
    print(" ")

In [ ]:
# Visualizing the outliers
# Calculating the number of rows and columns based on the number of numerical columns
num_columns = len(numerical_features)
num_rows = int(np.ceil(num_columns / 3))

# Creating a figure and axes array for subplots
fig, axes = plt.subplots(num_rows, 3, figsize=(15, 5 * num_rows))
axes = axes.flatten()

# Plotting boxplots for each numerical column
for i, column in enumerate(numerical_features):
    ax = axes[i]

    # Calculating IQR and bounds
    Q1 = data[column].quantile(0.25)
    Q3 = data[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    # Plotting the boxplot
    sns.boxplot(x=data[column], color='skyblue', flierprops=dict(marker='o', color='red', markersize=5), ax=ax)
    ax.axvline(lower_bound, color='red', linestyle='--', label='Lower Bound')
    ax.axvline(upper_bound, color='green', linestyle='--', label='Upper Bound')
    ax.set_title(f'Boxplot of {column}')
    ax.set_xlabel(column)
    ax.legend()
    ax.grid(axis='x', alpha=0.75)


for i in range(num_columns, len(axes)):
    axes[i].axis('off')

plt.tight_layout()
plt.show()

In [ ]:
# Function to handle outliers by capping
def handle_outliers(data, method='cap'):
    for column in data.columns:
        if column == 'YEAR':
            lower_bound, upper_bound = -0.86, 0.50
            data[column] = data[column].apply(lambda x: min(max(x, lower_bound), upper_bound) if method == 'cap' else x)

        elif column == 'QUANTITY':
            lower_bound, upper_bound = -0.31, 0.03
            data[column] = data[column].apply(lambda x: min(max(x, lower_bound), upper_bound) if method == 'cap' else x)

        elif column == 'VALUE_RS':
            lower_bound, upper_bound = -0.39, -0.19
            data[column] = data[column].apply(lambda x: min(max(x, lower_bound), upper_bound) if method == 'cap' else x)

    return data


# Applying the function to handle outliers
data_cleaned = handle_outliers(data.copy(), method='cap')

# Displaying the cleaned dataset
print(data_cleaned.head())

In [ ]:
# Step 16: Handling the multicollinearity in the numerical columns

# Computing the correlation for numerical features
correlation_matrix = data[numerical_features].corr()

# Displaying the correlation matrix
print(correlation_matrix)

In [ ]:
# Plotting the correlation heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f")
plt.title('Correlation Heatmap')
plt.show()

In [ ]:
# Step 17: Assignining the Target Variable

# Risk mapping for DESCRIPTION_03
risk_mapping = {
    'chlorine': 'Low',
    'nitric acid; sulphonitric acids': 'Medium',
    'cyanides and cyanide oxides: of sodium': 'High',
    'cyanides and cyanide oxides: other': 'High',
    'hydrogen peroxide, whether or not solidified with urea.': 'Medium',
    'iodine': 'Low',
    'bromine': 'Low',
    'chlorates and perchlorates; bromates and perbromates; iodates and periodates': 'Medium',
    'mineral or chemical fertilizers, nitrogenous': 'Low',
    'sulphuric acid; oleum': 'Medium',
    'halides and halide oxides of non metals': 'Low',
    'fluorides; fluorosilicates, fluoroaluminates, and other complex fluorine salts': 'Low'
}

# Define quantity thresholds
high_quantity_threshold = 10000
medium_quantity_threshold = 5000

# Function to calculate risk
def calculate_risk(description, quantity):
    description_risk = risk_mapping.get(description.lower(), 'Low')
    if description_risk == 'High' or quantity > high_quantity_threshold:
        return 'High'
    elif description_risk == 'Medium' or quantity > medium_quantity_threshold:
        return 'Medium'
    else:
        return 'Low'

# Applying the function
data['RISK'] = data.apply(
    lambda row: calculate_risk(row['DESCRIPTION_03'], row['QUANTITY']),
    axis=1
)

In [ ]:
# Visualizing the target variable
plt.figure(figsize=(8, 6))
sns.countplot(x='RISK', data=data)
plt.title('Risk Distribution')
plt.xlabel('Risk')
plt.ylabel('Count')
plt.show()

In [ ]:
# Step 18: Encoding the 'DESCRIPTION_03' column with label encoder and dropping the 'ENCODED_DESCRIPTION_03' column
label_encoder = LabelEncoder()
data['DESCRIPTION_03'] = label_encoder.fit_transform(data['DESCRIPTION_03'])
data.drop(columns=['ENCODED_DESCRIPTION_03'], inplace=True)

In [ ]:
# Step 19: Encoding the 'RISK' variable
label_encoder = LabelEncoder()
data['RISK'] = label_encoder.fit_transform(data['RISK'])

In [ ]:
# Step 20: Scaling the numerical features
scaler = StandardScaler()
data[numerical_features] = scaler.fit_transform(data[numerical_features])

In [ ]:
# Step 21: Splitting features and target
X = data.drop(columns=['RISK'])
y = data['RISK']

In [ ]:
# Step 22: Splitting data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Analyzing the distribution of the y train dataset
unique_values, counts = np.unique(y_train, return_counts=True)
for value, count in zip(unique_values, counts):
    print(f"Value: {value}, Count: {count}")

In [ ]:
# Visualizing the distribution of the target variable
sns.countplot(data=data, x='RISK')
plt.title('Target Variable Distribution')
plt.show()

In [ ]:
# Displaying the dataset shape
print(f"Shape of X_train: {X_train.shape}")
print(f"Shape of y_train: {y_train.shape}")

In [ ]:
# Display dataset information
print("Dataset Information:")
print(data.info())

In [ ]:
# Display dataset information
print("Dataset Information:")
print(data.head)

In [ ]:
# Handling the imbalance dataset

# Suppressing the FutureWarnings
warnings.filterwarnings("ignore", category=FutureWarning)

# Handling imbalanced dataset
smote = SMOTE(sampling_strategy='auto', random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)

print("Original dataset size:", X.shape)
print("Resampled dataset size:", X_resampled.shape)

In [ ]:
# Plotting class distribution after SMOTE
sns.countplot(x=y_resampled, palette="viridis")
plt.title("Class Distribution After SMOTE")
plt.xlabel("Class")
plt.ylabel("Count")
plt.show()

In [ ]:
# Displaying the dataset shape after applying the SMOTE
print(f"Shape of X_resampled: {X_resampled.shape}")
print(f"Shape of y_resampled: {y_resampled.shape}")

In [ ]:
# Standardizing the features
scaler = StandardScaler()
X_resampled_scaled = scaler.fit_transform(X_resampled)

In [ ]:
# Splitting data
X_train, X_test, y_train, y_test = train_test_split(X_resampled_scaled, y_resampled, test_size=0.2, stratify = y_resampled, random_state=42)

In [ ]:
# Display dataset information
print("Dataset Information:")
print(data.info())

In [ ]:
# Scaling the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [ ]:
# Splitting the data into training (60%) and temporary (40%) sets
X_train_scaled, X_temp, y_train, y_temp = train_test_split(X_scaled, y, test_size=0.4, random_state=42, stratify=y)

# Splitting the temporary set into validation (50% of temp, i.e., 20% of total) and test (50% of temp, i.e., 20% of total)
X_val_scaled, X_test_scaled, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp)

In [ ]:
# Standard scaling for the splits
X_train_scaled = scaler.fit_transform(X_train_scaled)
X_val_scaled = scaler.transform(X_val_scaled)
X_test_scaled = scaler.transform(X_test_scaled)

In [ ]:
# Setting a global seed for reproducibility
def set_seed(seed_value=42):
    np.random.seed(seed_value)
    random.seed(seed_value)
    tf.random.set_seed(seed_value)

# Defining the LSTM model
def create_lstm_model(input_shape, seed_value=42):
    # Setting the seed before building the model
    set_seed(seed_value)

    model = Sequential()
    model.add(LSTM(units=64, return_sequences=False, input_shape=input_shape))
    model.add(Dropout(0.2))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(1, activation='linear'))
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

In [ ]:
# Reshaping input data for LSTM (3D input required: samples, timesteps, features)
X_train_lstm = X_train_scaled.reshape(X_train_scaled.shape[0], 1, X_train_scaled.shape[1])
X_val_lstm = X_val_scaled.reshape(X_val_scaled.shape[0], 1, X_val_scaled.shape[1])
X_test_lstm = X_test_scaled.reshape(X_test_scaled.shape[0], 1, X_test_scaled.shape[1])

# Creating and train the LSTM model
lstm_model = create_lstm_model((X_train_lstm.shape[1], X_train_lstm.shape[2]))
lstm_model.fit(X_train_lstm, y_train, epochs=50, batch_size=32, validation_data=(X_val_lstm, y_val))

# Extracting LSTM features (use LSTM's output as features for Gradient Boosting)
lstm_train_features = lstm_model.predict(X_train_lstm)
lstm_val_features = lstm_model.predict(X_val_lstm)
lstm_test_features = lstm_model.predict(X_test_lstm)

In [ ]:
# Reshape features for Gradient Boosting
lstm_train_features = lstm_train_features.reshape(-1, 1)
lstm_val_features = lstm_val_features.reshape(-1, 1)
lstm_test_features = lstm_test_features.reshape(-1, 1)

In [ ]:
# Hyperparameter tuning for Gradient Boosting using GridSearchCV
param_grid = {
    'n_estimators': [50, 100, 150],
    'learning_rate': [0.01, 0.05, 0.1],
    'max_depth': [3, 5, 7],
    'min_samples_split': [2, 5, 10]
}

# Grid search cross-validation for Gradient Boosting
grid_search = GridSearchCV(GradientBoostingClassifier(), param_grid, cv=5, scoring='accuracy', n_jobs=-1, verbose=1)
grid_search.fit(lstm_train_features, y_train)

# Best parameters from grid search
print("Best Parameters for Gradient Boosting: ", grid_search.best_params_)

# Train Gradient Boosting on the best parameters
gb_model_class = grid_search.best_estimator_


Fitting 5 folds for each of 81 candidates, totalling 405 fits
Best Parameters for Gradient Boosting:  {'learning_rate': 0.01, 'max_depth': 3, 'min_samples_split': 10, 'n_estimators': 50}


In [ ]:
# Cross-validation on Gradient Boosting
cv_scores = cross_val_score(gb_model_class, lstm_train_features, y_train, cv=StratifiedKFold(5), scoring='accuracy')
print(f"Cross-validation Accuracy: {np.mean(cv_scores)} ± {np.std(cv_scores)}")

Cross-validation Accuracy: 0.9696 ± 0.005986651818838311


In [ ]:
# Predicting on test data
y_pred_class = gb_model_class.predict(lstm_test_features)

# Evaluation for classification
print("\nClassification Evaluation:")
print("Accuracy: ", accuracy_score(y_test, y_pred_class))
print("Classification Report:\n", classification_report(y_test, y_pred_class))

In [ ]:
# Generate the confusion matrix
cm = confusion_matrix(y_test, y_pred_class)

# Plotting the confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", cbar=False, xticklabels=['Class 0', 'Class 1', 'Class 2'], yticklabels=['Class 0', 'Class 1', 'Class 2'])
plt.title('Confusion Matrix')
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.show()

In [ ]:
# Overfitting Score: Train vs Test Accuracy
y_pred_train = gb_model_class.predict(lstm_train_features)
train_accuracy = accuracy_score(y_train, y_pred_train)
print("Train Accuracy: ", train_accuracy)

y_pred_test = gb_model_class.predict(lstm_test_features)
test_accuracy = accuracy_score(y_test, y_pred_test)
print("Test Accuracy: ", test_accuracy)

# Check for overfitting
if train_accuracy - test_accuracy > 0.1:
    print("The model might be overfitting. Consider regularization, reducing model complexity, or adding more data.")
else:
    print("The model has balanced performance on train and test sets.")

In [ ]:
# Saving Scaler
joblib.dump(scaler, '/content/drive/MyDrive/Comprehensive_Chemical_Risk_Prediction_Model/scaler.pkl')

# Saving LSTM Model
lstm_model.save('/content/drive/MyDrive/Comprehensive_Chemical_Risk_Prediction_Model/lstm_model.h5')

# Saving Gradient Boosting Model
joblib.dump(gb_model_class, '/content/drive/MyDrive/Comprehensive_Chemical_Risk_Prediction_Model/gb_model_class.pkl')

print("Models saved successfully!")

In [ ]:
# Loading the Scaler
scaler = joblib.load('/content/drive/MyDrive/Comprehensive_Chemical_Risk_Prediction_Model/scaler.pkl')

# Loading the LSTM Model
lstm_model = load_model('/content/drive/MyDrive/Comprehensive_Chemical_Risk_Prediction_Model/lstm_model.h5')

# Loading the Gradient Boosting Model
gb_model_class = joblib.load('/content/drive/MyDrive/Comprehensive_Chemical_Risk_Prediction_Model/gb_model_class.pkl')

In [252]:
from flask import Flask, request, jsonify
import numpy as np
import joblib
from keras.models import load_model
from sklearn.preprocessing import LabelEncoder, StandardScaler

app = Flask(__name__)

# Define encoding lists
importer_list = ["Global Traders Inc.", "Eastern Imports Ltd.", "Pacific Exports Co.",
                 "Summit Commerce", "EverGreen Enterprises"]
month_list = ["January", "February", "March", "April", "May", "June", "July", "August", "September", "October", "November", "December"]
country_list = ["USA", "INDIA", "CHINA", "VIETNAM", "RUSSIA", "KENYA", "BELGIUM", "SOUTH KOREA",
                "UKRAINE", "GERMANY", "COLOMBIA", "BANGLADESH", "SLOVENIA", "MALAYSIA", "US VIRGIN ISLANDS",
                "URUGUAY", "THAILAND", "JAPAN", "SRI LANKA", "GERMENY", "CAMBODIA", "ISRAEL", "SWITZERLAND",
                "UK", "CANADA", "TAIWAN", "TURKMENISTAN", "CHILE", "SINGAPORE", "FRANCE", "INDONESIA",
                "KAZAKHSTAN", "NIGERIA", "SOUTH AFRICA", "UAE", "SPAIN", "SAUDI ARABIA", "ARGENTINA", "BRAZIL",
                "ITALY", "EGYPT", "SLOVAKIA", "ROMANIA", "AUSTRIA", "MEXICO", "DENMARK", "PAKISTAN", "NETHERLANDS (HOLAND)", "GEORGIA"]
unit_list = ["KG", "PCS", "NOS", "GMS", "MTS", "LTR", "DRM"]

# Initialize encoders for categorical values
importer_encoder = LabelEncoder().fit(importer_list)
month_encoder = LabelEncoder().fit(month_list)
country_encoder = LabelEncoder().fit(country_list)
unit_encoder = LabelEncoder().fit(unit_list)

# Load pre-trained models and scalers
scaler = joblib.load('scaler.pkl')
lstm_model = load_model('lstm_model.h5')
gb_model_class = joblib.load('gb_model.pkl')

# Risk mapping dictionary
risk_mapping = {
    'chlorine': 'Low',
    'nitric acid; sulphonitric acids': 'Medium',
    'cyanides and cyanide oxides: of sodium': 'High',
    'hydrogen peroxide, whether or not solidified with urea.': 'Medium',
    'iodine': 'Low',
    'bromine': 'Low',
    'chlorates and perchlorates; bromates and perbromates; iodates and periodates': 'Medium',
    'mineral or chemical fertilizers, nitrogenous': 'Low',
    'sulphuric acid; oleum': 'Medium',
    'halides and halide oxides of non metals': 'Low',
    'fluorides; fluorosilicates, fluoroaluminates, and other complex fluorine salts': 'Low'
}

# Define quantity thresholds
high_quantity_threshold = 10000
medium_quantity_threshold = 5000

# Function to encode the inputs
def encode_inputs(importer_input, month_input, country_input, unit_input, year_input, hscode_input, description_3_input, quantity_input, value_input):
    try:
        importer_encoded = importer_encoder.transform([importer_input])[0] if importer_input in importer_list else len(importer_list)
        month_encoded = month_encoder.transform([month_input])[0] if month_input in month_list else len(month_list)
        country_encoded = country_encoder.transform([country_input])[0] if country_input in country_list else len(country_list)
        unit_encoded = unit_encoder.transform([unit_input])[0] if unit_input in unit_list else len(unit_list)

        return (importer_encoded, month_encoded, country_encoded, unit_encoded, year_input, hscode_input, len(description_3_input), quantity_input, value_input)
    except Exception as e:
        raise ValueError(f"Encoding failed: {e}")

# Function to calculate risk
def calculate_risk(description, quantity):
    description_risk = risk_mapping.get(description.lower(), 'Low')
    if description_risk == 'High' or quantity > high_quantity_threshold:
        return 'High'
    elif description_risk == 'Medium' or quantity > medium_quantity_threshold:
        return 'Medium'
    else:
        return 'Low'

# API endpoint for risk assessment
@app.route('/check-risk', methods=['POST'])
def check_risk():
    try:
        # Get JSON data from the request
        data = request.get_json()

        # Extract input data
        importer_input = data.get('importer')
        month_input = data.get('month')
        country_input = data.get('country')
        description_3_input = data.get('description')
        quantity_input = data.get('quantity', 0)
        value_input = data.get('value', 0)

        # Validate inputs
        if not all([importer_input, month_input, country_input, description_3_input]):
            return jsonify({'error': 'Missing required inputs'}), 400

        # Encode inputs
        encoded_values = encode_inputs(importer_input, month_input, country_input, "KG", 2025, 28080000, description_3_input, quantity_input, value_input)

        # Preprocess features for models
        features = np.array(encoded_values).reshape(1, -1)
        features_scaled = scaler.transform(features)

        # Prepare LSTM input format
        features_lstm = features_scaled.reshape(features_scaled.shape[0], 1, features_scaled.shape[1])

        # LSTM feature extraction
        lstm_features = lstm_model.predict(features_lstm)
        lstm_features = lstm_features.reshape(-1, 1)

        # Risk classification using Gradient Boosting
        prediction = gb_model_class.predict(lstm_features)
        classification = 'risky' if prediction == 1 else 'not risky'

        # Risk category assessment
        risk_category = calculate_risk(description_3_input, quantity_input)

        # Return the response
        return jsonify({
            'risk': risk_category,
            'classification': classification
        })

    except Exception as e:
        return jsonify({'error': str(e)}), 500

# Run the app
if __name__ == '__main__':
    app.run(debug=True)


FileNotFoundError: [Errno 2] No such file or directory: 'scaler.pkl'

In [ ]:
# Step 7: Encoding the Categorical Columns

# List of categorical columns to encode
encoding_columns = ['IMPORTER', 'MONTH', 'COUNTRY', 'HSCODE', 'UNIT', 'DESCRIPTION_01', 'DESCRIPTION_02', 'DESCRIPTION_03']

# Define the mapping dictionaries
importer_map = {
    "Global Traders Inc.": 0,
    "Eastern Imports Ltd.": 1,
    "Pacific Exports Co.": 2,
    "Summit Commerce": 3,
    "EverGreen Enterprises": 4
}

month_map = {
    "January": 0,
    "February": 1,
    "March": 2,
    "April": 3,
    "May": 4,
    "June": 5,
    "July": 6,
    "August": 7,
    "September": 8,
    "October": 9,
    "November": 10,
    "December": 11
}

country_map = {
    "USA": 0, "INDIA": 1, "CHINA": 2, "VIETNAM": 3, "RUSSIA": 4, "KENYA": 5, "BELGIUM": 6,
    "SOUTH KOREA": 7, "UKRAINE": 8, "GERMANY": 9, "COLOMBIA": 10, "BANGLADESH": 11,
    "SLOVENIA": 12, "MALAYSIA": 13, "US VIRGIN ISLANDS": 14, "URUGUAY": 15, "THAILAND": 16,
    "JAPAN": 17, "SRI LANKA": 18, "GERMENY": 19, "CAMBODIA": 20, "ISRAEL": 21, "SWITZERLAND": 22,
    "UK": 23, "CANADA": 24, "TAIWAN": 25, "TURKMENISTAN": 26, "CHILE": 27, "SINGAPORE": 28,
    "FRANCE": 29, "INDONESIA": 30, "KAZAKHSTAN": 31, "NIGERIA": 32, "SOUTH AFRICA": 33,
    "UAE": 34, "SPAIN": 35, "SAUDI ARABIA": 36, "ARGENTINA": 37, "BRAZIL": 38, "ITALY": 39,
    "EGYPT": 40, "SLOVAKIA": 41, "ROMANIA": 42, "AUSTRIA": 43, "MEXICO": 44, "DENMARK": 45,
    "PAKISTAN": 46, "NETHERLANDS (HOLAND)": 47, "GEORGIA": 48
}

unit_map = {
    "KG": 0,
    "PCS": 1,
    "NOS": 2,
    "GMS": 3,
    "MTS": 4,
    "LTR": 5,
    "DRM": 6
}

# DESCRIPTION_03 Encoding Map
description_03_map = {
    'chlorine': 0,
    'nitric acid; sulphonitric acids': 1,
    'cyanides and cyanide oxides: of sodium': 2,
    'hydrogen peroxide, whether or not solidified with urea.': 3,
    'iodine': 4,
    'bromine': 5,
    'chlorates and perchlorates; bromates and perbromates; iodates and periodates': 6,
    'mineral or chemical fertilizers, nitrogenous': 7,
    'sulphuric acid; oleum': 8,
    'halides and halide oxides of non metals': 9,
    'fluorides; fluorosilicates, fluoroaluminates, and other complex fluorine salts': 10,
    'cyanides, cyanide oxides and complex cyanides': 11,
}

# Mappings for DESCRIPTION_01
description_01_map = {
    'Inorganic chemicals; Organic or inorganic compounds of precious metals, of rare earth metals, of radioactive elements or of isotopes': 0,
    'Fertilizers': 1,
}

# Mappings for DESCRIPTION_02
description_02_map = {
    'Nitric acid; sulphonitric acids': 0,
    'Chlorates and perchlorates; bromates and perbromates; iodates and periodates': 1,
    'Mineral or chemical fertilizers, nitrogenous': 2,
    'Sulphuric acid; oleum': 3,
    'Halides and halide oxides of non metals': 4,
    'Fluorides; fluorosilicates, fluoroaluminates, and other complex fluorine salts': 5,
    'Hydrogen peroxide, whether or not solidified with urea': 6,
    'Fluorine, chlorine, bromine and iodine': 7,
    'Cyanides, cyanide oxides and complex cyanides': 8,
}

# Applying mappings manually for specific columns
data['IMPORTER'] = data['IMPORTER'].map(importer_map)
data['MONTH'] = data['MONTH'].map(month_map)
data['COUNTRY'] = data['COUNTRY'].map(country_map)
data['UNIT'] = data['UNIT'].map(unit_map)
data['DESCRIPTION_01'] = data['DESCRIPTION_01'].map(description_01_map)
data['DESCRIPTION_02'] = data['DESCRIPTION_02'].map(description_02_map)

# Encoding `DESCRIPTION_03` separately using the pre-defined map
data['ENCODED_DESCRIPTION_03'] = data['DESCRIPTION_03'].map(description_03_map)
print(data[['DESCRIPTION_03', 'ENCODED_DESCRIPTION_03']])
print(" ")

# Encoding 'HSCODE' column separately
data['HSCODE'] = label_encoder.fit_transform(data['HSCODE'])
print("Encoded column: HSCODE")
print(data['HSCODE'])
print(" ")

# Display all the encoded columns
print("All Encoded Columns:")
print(data)
